In [1]:
!pip install timm
!pip install lightning==2.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import timm
import torch

# Create the EfficientNetV2 model architecture
model = timm.create_model('efficientvit_m3', pretrained=True, num_classes= 8)  # Create the model without pretrained weights

filepath = 'drive/MyDrive/kvasir_GI_image/efficientvit_m3/lightning_logs/version_0/checkpoints/epoch=4-step=900.ckpt'
# Load the checkpoint
checkpoint = torch.load(filepath)

# Map checkpoint keys to model keys and load compatible layers
model_state_dict = model.state_dict()
for key in checkpoint.keys():
    if key in model_state_dict:
        model_state_dict[key] = checkpoint[key]

# Load the modified state_dict into the model's layers
model.load_state_dict(model_state_dict)


<All keys matched successfully>

In [3]:
from torchsummary import summary

summary(model,(3,224,224),device = "cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 112, 112]             432
       BatchNorm2d-2         [-1, 16, 112, 112]              32
              ReLU-3         [-1, 16, 112, 112]               0
            Conv2d-4           [-1, 32, 56, 56]           4,608
       BatchNorm2d-5           [-1, 32, 56, 56]              64
              ReLU-6           [-1, 32, 56, 56]               0
            Conv2d-7           [-1, 64, 28, 28]          18,432
       BatchNorm2d-8           [-1, 64, 28, 28]             128
              ReLU-9           [-1, 64, 28, 28]               0
           Conv2d-10          [-1, 128, 14, 14]          73,728
      BatchNorm2d-11          [-1, 128, 14, 14]             256
         Identity-12          [-1, 128, 14, 14]               0
           Conv2d-13          [-1, 128, 14, 14]           1,152
      BatchNorm2d-14          [-1, 128,

In [7]:
new_head = torch.nn.Sequential()
model.head = new_head

In [8]:
from torchsummary import summary

summary(model,(3,224,224),device = "cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 112, 112]             432
       BatchNorm2d-2         [-1, 16, 112, 112]              32
              ReLU-3         [-1, 16, 112, 112]               0
            Conv2d-4           [-1, 32, 56, 56]           4,608
       BatchNorm2d-5           [-1, 32, 56, 56]              64
              ReLU-6           [-1, 32, 56, 56]               0
            Conv2d-7           [-1, 64, 28, 28]          18,432
       BatchNorm2d-8           [-1, 64, 28, 28]             128
              ReLU-9           [-1, 64, 28, 28]               0
           Conv2d-10          [-1, 128, 14, 14]          73,728
      BatchNorm2d-11          [-1, 128, 14, 14]             256
         Identity-12          [-1, 128, 14, 14]               0
           Conv2d-13          [-1, 128, 14, 14]           1,152
      BatchNorm2d-14          [-1, 128,

In [10]:
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import datasets

dataset_dir = '/content/drive/MyDrive/kvasir-dataset-v2'

image_width =224
image_height =224
image_size =(image_width, image_height)

mean= (0.485, 0.456, 0.406)
std= (0.229, 0.224, 0.225)

# Define data transformation for validation and test data
test_transform = transforms.Compose([
    transforms.Resize(image_size),                # Resize the image to 224x224
    transforms.ToTensor(),                 # Convert image to PyTorch tensor
    transforms.Normalize(mean, std)  # Normalize the tensor (ImageNet statistics)
])


In [ ]:
from PIL import Image
import os
import numpy as np

model.eval()

feature_dict = {}

for cls in os.listdir(dataset_dir):
  for img in os.listdir(os.path.join(dataset_dir,cls)):
    image_path = os.path.join(dataset_dir,cls,img)
    image = Image.open(image_path)
    image = test_transform(image)
    image = image.unsqueeze(0)
    # Make a forward pass to obtain predictions
    with torch.no_grad():
        output = model(image)

    features = output[-1]
    flatten_features = torch.nn.Flatten()(features).squeeze()

    # Add a linear layer to reduce the number of features to 2600
    linear_layer = torch.nn.Linear(len(flatten_features), 2600)
    final_features = linear_layer(flatten_features)

    # Convert the output tensor to a NumPy array
    output_array = final_features.detach().numpy()

    # Append the class (cls) to the last index of the array
    output_array = np.append(output_array, cls)

    feature_dict[img] = output_array


In [25]:
from PIL import Image
import os
import numpy as np

model.eval()

feature_dict = {}
label = []
feature_list = []

for cls in os.listdir(dataset_dir):
  for img in os.listdir(os.path.join(dataset_dir,cls)):
    image_path = os.path.join(dataset_dir,cls,img)
    image = Image.open(image_path)
    image = test_transform(image)
    image = image.unsqueeze(0)
    # Make a forward pass to obtain predictions
    with torch.no_grad():
        output = model(image)

    # features = output[-1]
    flatten_features = torch.nn.Flatten()(output).squeeze()

    # # Add a linear layer to reduce the number of features to 2600
    # linear_layer = torch.nn.Linear(len(flatten_features), 2600)
    # final_features = linear_layer(flatten_features)

    # Convert the output tensor to a NumPy array
    output_array = flatten_features.detach().numpy()

    feature_list.append(output_array)
    label.append(cls)
    # Append the class (cls) to the last index of the array
    # output_array = np.append(output_array, cls)

    # feature_dict['img'] = output_array

In [27]:
feature_list = np.array(feature_list)
feature_list.shape

(8000, 320)

In [28]:
feature_dict = {'features': feature_list, 'label': label}

In [31]:
import pickle

filepath = 'drive/MyDrive/feature/diganto_efficientvit_m3_320.pkl'

# Save the dictionary into a pickle file
with open(filepath, 'wb') as f:
    pickle.dump(feature_dict, f)

In [32]:
import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [35]:
!gsutil ls gs://tibot-ml-labeling/datasets/**

gs://tibot-ml-labeling/datasets/Humanbody/Diganto/dataset.zip
gs://tibot-ml-labeling/datasets/Humanbody/gaju/HbodyDataset.zip
gs://tibot-ml-labeling/datasets/Humanbody/rifat/human_dataset.zip
gs://tibot-ml-labeling/datasets/cats_vs_dogs_dataset.zip
gs://tibot-ml-labeling/datasets/kvasir/diganto_efficientvit_m3_2600.pkl
gs://tibot-ml-labeling/datasets/kvasir/diganto_efficientvit_m3_320.pkl
gs://tibot-ml-labeling/datasets/kvasir/gaju_fastvit_t8_6144.pkl
gs://tibot-ml-labeling/datasets/kvasir/rifat_mobileOne_s4_2048.pkl


In [34]:
!gsutil -mq cp {filepath} gs://tibot-ml-labeling/datasets/kvasir/